## Imbalance Data

In [1]:
import tensorflow as tf
import numpy as np
from imblearn.over_sampling import BorderlineSMOTE
import random

/home/chanseok/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [7]:
EPOCHS = 100

## Model Define

In [3]:
class MyModel(tf.keras.Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.flatten = tf.keras.layers.Flatten()
        self.dense1 = tf.keras.layers.Dense(1024, activation='relu')
        self.dense2 = tf.keras.layers.Dense(1, activation='sigmoid')
        
    def call(self, x, training=False, mask=None):
        x = self.flatten(x)
        x = self.dense1(x)
        return self.dense2(x)

## Prepare Dataset

In [11]:
cifar10 = tf.keras.datasets.cifar10

(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# Convert imbalanced data
x_train_small = []
y_train_small = []
for x, y in zip(x_train, y_train):
    if (y == 0 and random.randint(0, 100) < 10) or y == 1:
        x_train_small.append(x[:])
        y_train_small.append(y)
        
x_test_small = []
y_test_small = []
for x, y in zip(x_test, y_test):
    if (y == 0) or (y == 1):
        x_test_small.append(x[:])
        y_test_small.append(y)
        
x_train = np.stack(x_train_small, axis=0).astype(np.float32)
y_train = np.stack(y_train_small, axis=0).astype(np.float32)

x_test = np.stack(x_test_small, axis=0).astype(np.float32)
y_test = np.stack(y_test_small, axis=0).astype(np.float32)

train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(32).prefetch(2048)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32).prefetch(2048)

## Model Training with Keras API

In [12]:
model = MyModel()
model.compile(optimizer='adam', 
              loss = 'binary_crossentropy', 
             metrics=['accuracy',
                      tf.keras.metrics.Precision(name='precision'),
                      tf.keras.metrics.Recall(name='recall')])
model.fit(train_ds, validation_data=test_ds, epochs=EPOCHS);

Epoch 1/100
174/174 [==============================] - 1s 4ms/step - loss: 0.4767 - accuracy: 0.8862 - precision: 0.9184 - recall: 0.9590 - val_loss: 0.4628 - val_accuracy: 0.7735 - val_precision: 0.8140 - val_recall: 0.7090
Epoch 2/100
174/174 [==============================] - 0s 2ms/step - loss: 0.3009 - accuracy: 0.9004 - precision: 0.9254 - recall: 0.9676 - val_loss: 1.8609 - val_accuracy: 0.5030 - val_precision: 0.5015 - val_recall: 1.0000
Epoch 3/100
174/174 [==============================] - 0s 2ms/step - loss: 0.2559 - accuracy: 0.9075 - precision: 0.9274 - recall: 0.9736 - val_loss: 0.7059 - val_accuracy: 0.6715 - val_precision: 0.6059 - val_recall: 0.9810
Epoch 4/100
174/174 [==============================] - 0s 2ms/step - loss: 0.2334 - accuracy: 0.9167 - precision: 0.9315 - recall: 0.9796 - val_loss: 0.5348 - val_accuracy: 0.7515 - val_precision: 0.6755 - val_recall: 0.9680
Epoch 5/100
174/174 [==============================] - 0s 2ms/step - loss: 0.2213 - accuracy: 0.9212

174/174 [==============================] - 0s 2ms/step - loss: 0.0864 - accuracy: 0.9665 - precision: 0.9763 - recall: 0.9868 - val_loss: 0.7952 - val_accuracy: 0.7690 - val_precision: 0.6871 - val_recall: 0.9880
Epoch 38/100
174/174 [==============================] - 0s 2ms/step - loss: 0.0967 - accuracy: 0.9628 - precision: 0.9730 - recall: 0.9862 - val_loss: 0.4589 - val_accuracy: 0.8660 - val_precision: 0.8194 - val_recall: 0.9390
Epoch 39/100
174/174 [==============================] - 0s 2ms/step - loss: 0.0829 - accuracy: 0.9670 - precision: 0.9768 - recall: 0.9868 - val_loss: 0.4926 - val_accuracy: 0.8480 - val_precision: 0.7915 - val_recall: 0.9450
Epoch 40/100
174/174 [==============================] - 0s 2ms/step - loss: 0.0739 - accuracy: 0.9704 - precision: 0.9779 - recall: 0.9896 - val_loss: 0.5374 - val_accuracy: 0.8510 - val_precision: 0.7930 - val_recall: 0.9500
Epoch 41/100
174/174 [==============================] - 0s 2ms/step - loss: 0.0763 - accuracy: 0.9699 - preci

174/174 [==============================] - 0s 2ms/step - loss: 0.0236 - accuracy: 0.9924 - precision: 0.9956 - recall: 0.9960 - val_loss: 1.6176 - val_accuracy: 0.7455 - val_precision: 0.6647 - val_recall: 0.9910
Epoch 74/100
174/174 [==============================] - 0s 2ms/step - loss: 0.0271 - accuracy: 0.9919 - precision: 0.9944 - recall: 0.9966 - val_loss: 1.0549 - val_accuracy: 0.8130 - val_precision: 0.7400 - val_recall: 0.9650
Epoch 75/100
174/174 [==============================] - 0s 2ms/step - loss: 0.0365 - accuracy: 0.9877 - precision: 0.9924 - recall: 0.9940 - val_loss: 0.8683 - val_accuracy: 0.8230 - val_precision: 0.7580 - val_recall: 0.9490
Epoch 76/100
174/174 [==============================] - 0s 2ms/step - loss: 0.0470 - accuracy: 0.9825 - precision: 0.9886 - recall: 0.9920 - val_loss: 1.0662 - val_accuracy: 0.7945 - val_precision: 0.7199 - val_recall: 0.9640
Epoch 77/100
174/174 [==============================] - 0s 2ms/step - loss: 0.0296 - accuracy: 0.9903 - preci

## BorderlineSMOTE

In [9]:
smote = BorderlineSMOTE()
x_train = x_train.reshape((x_train.shape[0], x_train.shape[1] * x_train.shape[2] * x_train.shape[3])).astype(np.float32)
x_test = x_test.reshape((x_test.shape[0], x_test.shape[1] * x_test.shape[2] * x_test.shape[3])).astype(np.float32)

x_train, y_train = smote.fit_resample(x_train, y_train)

train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(32).prefetch(2048)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32).prefetch(2048)

In [10]:
model = MyModel()
model.compile(optimizer='adam', 
              loss = 'binary_crossentropy', 
             metrics=['accuracy',
                      tf.keras.metrics.Precision(name='precision'),
                      tf.keras.metrics.Recall(name='recall')])
model.fit(train_ds, validation_data=test_ds, epochs=EPOCHS);

Epoch 1/100
313/313 [==============================] - 1s 3ms/step - loss: 0.5807 - accuracy: 0.7535 - precision: 0.7775 - recall: 0.7102 - val_loss: 0.4665 - val_accuracy: 0.7855 - val_precision: 0.7574 - val_recall: 0.8400
Epoch 2/100
313/313 [==============================] - 1s 2ms/step - loss: 0.3934 - accuracy: 0.8371 - precision: 0.8713 - recall: 0.7910 - val_loss: 0.4205 - val_accuracy: 0.8155 - val_precision: 0.8725 - val_recall: 0.7390
Epoch 3/100
313/313 [==============================] - 1s 2ms/step - loss: 0.3199 - accuracy: 0.8685 - precision: 0.9008 - recall: 0.8282 - val_loss: 0.4099 - val_accuracy: 0.8285 - val_precision: 0.8150 - val_recall: 0.8500
Epoch 4/100
313/313 [==============================] - 1s 2ms/step - loss: 0.2668 - accuracy: 0.8939 - precision: 0.9227 - recall: 0.8598 - val_loss: 0.3983 - val_accuracy: 0.8445 - val_precision: 0.8335 - val_recall: 0.8610
Epoch 5/100
313/313 [==============================] - 1s 2ms/step - loss: 0.2336 - accuracy: 0.9085

313/313 [==============================] - 1s 2ms/step - loss: 0.0306 - accuracy: 0.9898 - precision: 0.9936 - recall: 0.9860 - val_loss: 1.2753 - val_accuracy: 0.8045 - val_precision: 0.7277 - val_recall: 0.9730
Epoch 38/100
313/313 [==============================] - 1s 2ms/step - loss: 0.0465 - accuracy: 0.9844 - precision: 0.9879 - recall: 0.9808 - val_loss: 0.9425 - val_accuracy: 0.8435 - val_precision: 0.7818 - val_recall: 0.9530
Epoch 39/100
313/313 [==============================] - 1s 2ms/step - loss: 0.0344 - accuracy: 0.9880 - precision: 0.9907 - recall: 0.9852 - val_loss: 1.2441 - val_accuracy: 0.8175 - val_precision: 0.7437 - val_recall: 0.9690
Epoch 40/100
313/313 [==============================] - 1s 2ms/step - loss: 0.0251 - accuracy: 0.9923 - precision: 0.9952 - recall: 0.9894 - val_loss: 1.0205 - val_accuracy: 0.8415 - val_precision: 0.7774 - val_recall: 0.9570
Epoch 41/100
313/313 [==============================] - 1s 2ms/step - loss: 0.0326 - accuracy: 0.9896 - preci

313/313 [==============================] - 1s 2ms/step - loss: 0.0097 - accuracy: 0.9966 - precision: 0.9974 - recall: 0.9958 - val_loss: 1.2358 - val_accuracy: 0.8525 - val_precision: 0.7940 - val_recall: 0.9520
Epoch 74/100
313/313 [==============================] - 1s 2ms/step - loss: 0.0117 - accuracy: 0.9961 - precision: 0.9974 - recall: 0.9948 - val_loss: 2.7452 - val_accuracy: 0.7085 - val_precision: 0.6337 - val_recall: 0.9880
Epoch 75/100
313/313 [==============================] - 1s 2ms/step - loss: 0.0241 - accuracy: 0.9927 - precision: 0.9934 - recall: 0.9920 - val_loss: 1.1588 - val_accuracy: 0.8600 - val_precision: 0.8136 - val_recall: 0.9340
Epoch 76/100
313/313 [==============================] - 1s 2ms/step - loss: 0.0178 - accuracy: 0.9946 - precision: 0.9956 - recall: 0.9936 - val_loss: 1.5472 - val_accuracy: 0.8240 - val_precision: 0.7512 - val_recall: 0.9690
Epoch 77/100
313/313 [==============================] - 1s 2ms/step - loss: 0.0131 - accuracy: 0.9956 - preci